# /!\ WARNING

 - *Mel Spectrogram* extracted of 431 element **BUT** *stack* truncate to **427**
 - The three models runned on gondor are for the first.
 
 - The features are being extracted with the following parameters
  - *eml spectrogram* no change
  - *stack* 500 elements

# Mix
## Double, independant, convolution for the melspectrogram and the "stack"

***
For each model, the following teste will be done:
 - classic GRU with basic threshold, hysteresis, and primitive based localization
 - Weighted GRU (0.10, 0.20, 0.23, 0.25, 0.30, 0.5) with the same localization methods


In [1]:
# Load the required modules
import sys
sys.path.append("../..")
from datasetGenerator import DCASE2018
from Encoder import Encoder
from Binarizer import Binarizer

from keras.layers import GRU, Bidirectional, Layer, TimeDistributed, Dense, GRUCell
from keras.models import model_from_json, Model
from keras import backend as K
from keras import regularizers
import matplotlib.pyplot as plt
import matplotlib.patches as patch
from subprocess import call

# evaluate
from evaluation_measures import event_based_evaluation
from dcase_util.containers import MetaDataContainer

import librosa.display
import librosa
import numpy as np
import os
import copy

Using TensorFlow backend.


# Load the dataset

In [9]:
# Load the test data create the test dataset
# load the file list
featTestPath = "/homeLocal/eriador/Documents/Corpus/DCASE2018/features_2/test/"
featTestList = os.listdir(os.path.join(featTestPath, "mel"))

# load the meta data ----
metaPath = "/homeLocal/eriador/Documents/Corpus/DCASE2018/meta/test.csv"
with open(metaPath, "r") as metaFile:
    metadata = metaFile.read().splitlines()[1:]
    
metadata = [i.split("\t") for i in metadata]

# load the features
featTest = {"mel": [], "stack": []}
for file in featTestList:
    pathMel = os.path.join(featTestPath, "mel", file)
    pathStack = os.path.join(featTestPath, "stack", file)
                          
    featureMel = np.load(pathMel)
    featureStack = np.load(pathStack)
        
    # preprocessing
    featureMel = np.expand_dims(featureMel, axis=-1)
    featureStack = np.expand_dims(featureStack, axis=-1)
    
    featTest["mel"].append(featureMel)
    featTest["stack"].append(featureStack)
    
featTest["mel"] = np.array(featTest["mel"])


# Load the models

In [3]:
modelJsonPath = "/homeLocal/eriador/Documents/DCASE2018/results/testing/mel_noGRU/experiment_noGRU_model.json"
modelWeightPath = "/homeLocal/eriador/Documents/DCASE2018/results/testing/mel_noGRU/experiment_noGRU_weight.h5py"

# Remove the GlobalAveragePooling1D to trace score per frames

In [12]:
##### load the model and perform prediction
def useClassic():
    K.clear_session()

    with open(modelJsonPath, "r") as modelJsonFile:
        model = model_from_json(modelJsonFile.read())
    model.load_weights(modelWeightPath)

    #model.summary()

    intermediate_model = Model(input=model.input, output=model.get_layer("time_distributed_1").output)
    #intermediate_model.summary()
    
    return intermediate_model

# retreive information about the custom
model = useClassic()
prediction = model.predict(featTest["mel"])

/homeLocal/eriador/.miniconda3/envs/dcase/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ti...)`
  # This is added back by InteractiveShellApp.init_path()


# Calculate the score using the **baseline tool**

In [13]:
def basic(method: str = "threshold", **kwargs):
    e = Encoder()
    segments = e.encode(prediction, method=method, **kwargs)
    evaluation = e.parse(segments, featTestList)
    
    # write the evaluation on the disk
    with open("perso_eval.csv", "w") as f:
        f.write("filename\tonset\toffset\tevent_label\n")
        f.write(evaluation)

    perso_event_list = MetaDataContainer()
    perso_event_list.load(filename="perso_eval.csv")

    ref_event_list = MetaDataContainer()
    ref_event_list.load(filename="../../meta/test.csv")

    event_based_metric = event_based_evaluation(ref_event_list, perso_event_list)

    print(event_based_metric)
    
def printTable(results: list):
    
    print("%-*s | %-*s | %-*s | %-*s | %-*s | %-*s | %-*s | %-*s | %-*s |" % (
        22, "model", 22, "Global F-measure", 22, "Global Precision", 22, "Global Recall", 22, "ER",
        22, "Class-wise F-measure", 22, "Class-wise Precision", 22, "Class-wise Recall", 22, "ER"))
    print(("-" * 22 + " | ") * 9)
    
    for weight, info in results:
        print("%-*s | " % (22, "CGRU %.2f" % (weight)), end="")
        print("%-*s | " % (22, "%.2f %%" % (info["overall"]["f_measure"]["f_measure"] * 100)), end="")
        print("%-*s | " % (22, "%.2f %%" % (info["overall"]["f_measure"]["precision"] * 100)), end="")
        print("%-*s | " % (22, "%.2f %%" % (info["overall"]["f_measure"]["recall"] * 100)), end="")
        print("%-*s | " % (22, "%.2f " % (info["overall"]["error_rate"]["error_rate"])), end="")
        print("%-*s | " % (22, "%.2f %%" % (info["class-wise"]["f_measure"]["f_measure"] * 100)), end="")
        print("%-*s | " % (22, "%.2f %%" % (info["class-wise"]["f_measure"]["precision"] * 100)), end="")
        print("%-*s | " % (22, "%.2f %%" % (info["class-wise"]["f_measure"]["recall"] * 100)), end="")
        print("%-*s | " % (22, "%.2f " % (info["class-wise"]["error_rate"]["error_rate"])), end="")
        print("")
        
def evaluate(evaluation):
    # write the evaluation on the disk
    print("perform evaluation ...")
    with open("perso_eval.csv", "w") as f:
        f.write("filename\tonset\toffset\tevent_label\n")
        f.write(evaluation)

    perso_event_list = MetaDataContainer()
    perso_event_list.load(filename="perso_eval.csv")

    ref_event_list = MetaDataContainer()
    ref_event_list.load(filename="../../meta/test.csv")

    event_based_metric = event_based_evaluation(ref_event_list, perso_event_list)
    #print(event_based_metric)

    result = {
        "overall": event_based_metric.results_overall_metrics(),
        "class-wise": event_based_metric.results_class_wise_average_metrics(),
    }

    return result
    

def advanced(temporal_weights: list, method="threshold", **kwargs) -> list:
    results = []
    
    # classic ----------------------
    print("loading model ...")
    model = useClassic()
    print("predicting ...")
    prediction = model.predict([featTest["mel"], featTest["stack"]])
    
    e = Encoder()
    print("encoding results ...")
    segments = e.encode(prediction, method=method, **kwargs)
    evaluation = e.parse(segments, featTestList)
    
    results.append((1.0, evaluate(evaluation)))
    
    # custom GRU prediction - ---------------------
    for weight in temporal_weights:
        print("loading model ...")
        model = useCustomGRU(weight)
        print("predicting ...")
        prediction = model.predict(featTest)
        
        e = Encoder()
        print("encoding results ...")
        segments = e.encode(prediction, method=method, **kwargs)
        evaluation = e.parse(segments, featTestList)
        
        results.append((weight, evaluate(evaluation)))
        
    return results
        
#print("HYSTERESIS")
#printTable( advanced([0.1, 0.2, 0.23, 0.25, 0.3, 0.5], method="hysteresis") )

#print("PRIMITIVE")
#printTable( advanced([0.1, 0.2, 0.23, 0.25, 0.3, 0.5], method="primitive", window_size=8, stride=1, thresshold=2) )

print("THRESHOLD")
printTable( advanced([0.1, 0.2, 0.23, 0.25, 0.3, 0.5]) )

#basic(method="hysteresis")
#basic(method="hysteresis", high=0.7, low=0.3)
#basic(method="hysteresis", high=0.8, low=0.2)
#basic(method="derivative")
#basic(method="primitive", window_size=10, stride=1, threshold=3)
basic()

Event based metrics (onset-offset)
  Evaluated length                  : 2616.88 sec
  Evaluated files                   : 288 
  Evaluate onset                    : True 
  Evaluate offset                   : True 
  T collar                          : 200.00 ms
  Offset (length)                   : 20.00 %

  Overall metrics (micro-average)
  F-measure
    F-measure (F1)                  : 3.59 %
    Precision                       : 1.97 %
    Recall                          : 20.42 %
  Error rate
    Error rate (ER)                 : 10.85 
    Substitution rate               : 0.13 
    Deletion rate                   : 0.67 
    Insertion rate                  : 10.05 

  Class-wise average metrics (macro-average)
  F-measure
    F-measure (F1)                  : 4.13 %
    Precision                       : 2.45 %
    Recall                          : 18.02 %
  Error rate
    Error rate (ER)                 : 15.52 
    Deletion rate                   : 0.82 
    Insertion rate  

# 20 ms frame length

***

## *Threshold based segmentation with "hole filling"*

 - **CGRU x.xx** : Custom Gate Recurent Unit usage with temporal weight of x.xx
 - **GRU** : The classic GRU implementation
 
model                  | Global F-measure       | Global Precision       | Global Recall          | ER                     | Class-wise F-measure   | Class-wise Precision   | Class-wise Recall      | ER                     |
---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- |
GRU                    | 3.79 %                 | 3.15 %                 | 4.75 %                 | 2.38                   | 4.96 %                 | 3.56 %                 | 11.99 %                | 3.25                   | 
CGRU 0.10              | 6.61 %                 | 4.88 %                 | 10.26 %                | 2.88                   | 5.55 %                 | 6.68 %                 | 7.25 %                 | 5.18                   | 
CGRU 0.20              | 9.60 %                 | 7.19 %                 | 14.46 %                | 2.70                   | 8.25 %                 | 9.82 %                 | 10.43 %                | 4.78                   | 
CGRU 0.23              | 10.41 %                | 7.88 %                 | 15.34 %                | 2.62                   | 9.40 %                 | 10.85 %                | 11.86 %                | 4.61                   | 
CGRU 0.25              | 10.59 %                | 8.06 %                 | 15.45 %                | 2.59                   | 9.88 %                 | 16.22 %                | 12.17 %                | 4.50                   | 
CGRU 0.30              | 9.45 %                 | 7.32 %                 | 13.36 %                | 2.54                   | 9.23 %                 | 10.04 %                | 12.03 %                | 4.30                   | 
CGRU 0.50              | 4.23 %                 | 3.39 %                 | 5.63 %                 | 2.52                   | 4.47 %                 | 4.07 %                 | 9.26 %                 | 3.68                   | 
Baseline | 14.06 % | - | - | 1.54 | - | - | - | - |



## *Hysteresis based segmentation*

model                  | Global F-measure       | Global Precision       | Global Recall          | ER                     | Class-wise F-measure   | Class-wise Precision   | Class-wise Recall      | ER                     |
---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | 
GRU                    | 4.52 %                 | 4.40 %                 | 4.64 %                 | 1.93                   | 6.40 %                 | 5.05 %                 | 11.75 %                | 2.48                   | 
CGRU 0.10              | 2.94 %                 | 2.26 %                 | 4.19 %                 | 2.76                   | 2.27 %                 | 2.02 %                 | 3.60 %                 | 4.55                   | 
CGRU 0.20              | 5.05 %                 | 4.04 %                 | 6.73 %                 | 2.52                   | 4.61 %                 | 4.98 %                 | 5.72 %                 | 4.17                   | 
CGRU 0.23              | 5.40 %                 | 4.37 %                 | 7.06 %                 | 2.46                   | 4.44 %                 | 5.00 %                 | 5.82 %                 | 4.04                   | 
CGRU 0.25              | 5.41 %                 | 4.42 %                 | 6.95 %                 | 2.43                   | 4.37 %                 | 5.13 %                 | 5.83 %                 | 3.93                   | 
CGRU 0.30              | 5.93 %                 | 5.00 %                 | 7.28 %                 | 2.30                   | 5.52 %                 | 7.95 %                 | 6.75 %                 | 3.67                   | 
CGRU 0.50              | 3.60 %                 | 3.23 %                 | 4.08 %                 | 2.15                   | 3.83 %                 | 3.51 %                 | 7.90 %                 | 3.06                   | 
Baseline | 14.06 % | - | - | 1.54 | - | - | - | - |



## *Primitive based segmentation*

model                  | Global F-measure       | Global Precision       | Global Recall          | ER                     | Class-wise F-measure   | Class-wise Precision   | Class-wise Recall      | ER                     |
---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | 
GRU                    | 5.11 %                 | 4.25 %                 | 6.40 %                 | 2.33                   | 6.29 %                 | 4.60 %                 | 15.28 %                | 3.22                   | 
CGRU 0.10              | 7.46 %                 | 5.48 %                 | 11.70 %                | 2.88                   | 7.13 %                 | 8.26 %                 | 9.00 %                 | 5.21                   | 
CGRU 0.20              | 9.29 %                 | 6.95 %                 | 14.02 %                | 2.72                   | 9.14 %                 | 13.17 %                | 12.08 %                | 4.70                   | 
CGRU 0.23              | 9.19 %                 | 6.91 %                 | 13.69 %                | 2.69                   | 9.12 %                 | 10.09 %                | 12.34 %                | 4.59                   | 
CGRU 0.25              | 8.59 %                 | 6.53 %                 | 12.58 %                | 2.65                   | 8.79 %                 | 9.25 %                 | 12.07 %                | 4.45                   | 
CGRU 0.30              | 6.25 %                 | 4.77 %                 | 9.05 %                 | 2.69                   | 6.76 %                 | 6.77 %                 | 9.82 %                 | 4.31                   | 
CGRU 0.50              | 3.63 %                 | 2.89 %                 | 4.86 %                 | 2.55                   | 4.18 %                 | 3.67 %                 | 9.91 %                 | 3.72                   | 
Baseline | 14.06 % | - | - | 1.54 | - | - | - | - |


# 40 ms frame length

***

## *Threshold based segmentation with "hole filling"*

 - **CGRU x.xx** : Custom Gate Recurent Unit usage with temporal weight of x.xx
 - **GRU** : The classic GRU implementation
 
model                  | Global F-measure       | Global Precision       | Global Recall          | ER                     | Class-wise F-measure   | Class-wise Precision   | Class-wise Recall      | ER                     |
---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | 
GRU                    | 4.31 %                 | 3.95 %                 | 4.75 %                 | 2.07                   | 6.68 %                 | 5.33 %                 | 11.40 %                | 2.55                   | 
CGRU 0.10              | 12.96 %                | 10.94 %                | 15.89 %                | 2.12                   | 11.47 %                | 12.47 %                | 10.94 %                | 3.73                   | 
CGRU 0.20              | 13.47 %                | 11.69 %                | 15.89 %                | 2.02                   | 11.35 %                | 11.81 %                | 12.29 %                | 3.37                   | 
CGRU 0.23              | 12.73 %                | 11.17 %                | 14.79 %                | 2.01                   | 11.37 %                | 11.47 %                | 12.50 %                | 3.22                   | 
CGRU 0.25              | 12.57 %                | 11.05 %                | 14.57 %                | 2.01                   | 11.52 %                | 11.52 %                | 12.94 %                | 3.17                   | 
CGRU 0.30              | 9.25 %                 | 8.07 %                 | 10.82 %                | 2.10                   | 8.24 %                 | 8.37 %                 | 10.58 %                | 3.18                   | 
CGRU 0.50              | 4.00 %                 | 3.52 %                 | 4.64 %                 | 2.19                   | 4.69 %                 | 4.34 %                 | 7.81 %                 | 2.92                   | 
Baseline | 14.06 % | - | - | 1.54 | - | - | - | - |



## *Hysteresis based segmentation*

model                  | Global F-measure       | Global Precision       | Global Recall          | ER                     | Class-wise F-measure   | Class-wise Precision   | Class-wise Recall      | ER                     |
---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | 
GRU                    | 5.43 %                 | 6.15 %                 | 4.86 %                 | 1.67                   | 8.90 %                 | 8.53 %                 | 11.18 %                | 1.90                   | 
CGRU 0.10              | 6.91 %                 | 7.10 %                 | 6.73 %                 | 1.81                   | 5.79 %                 | 9.08 %                 | 5.75 %                 | 2.60                   | 
CGRU 0.20              | 7.91 %                 | 8.51 %                 | 7.40 %                 | 1.71                   | 6.40 %                 | 7.08 %                 | 7.27 %                 | 2.38                   | 
CGRU 0.23              | 7.83 %                 | 8.46 %                 | 7.28 %                 | 1.71                   | 6.49 %                 | 7.02 %                 | 7.32 %                 | 2.35                   | 
CGRU 0.25              | 7.91 %                 | 8.50 %                 | 7.40 %                 | 1.71                   | 7.13 %                 | 7.47 %                 | 8.23 %                 | 2.30                   | 
CGRU 0.30              | 7.39 %                 | 8.20 %                 | 6.73 %                 | 1.68                   | 7.44 %                 | 8.28 %                 | 8.57 %                 | 2.20                   | 
CGRU 0.50              | 3.57 %                 | 3.88 %                 | 3.31 %                 | 1.76                   | 4.52 %                 | 3.93 %                 | 7.21 %                 | 2.20                   | 
Baseline | 14.06 % | - | - | 1.54 | - | - | - | - |



## *Primitive based segmentation*

model                  | Global F-measure       | Global Precision       | Global Recall          | ER                     | Class-wise F-measure   | Class-wise Precision   | Class-wise Recall      | ER                     |
---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | 
GRU                    | 5.50 %                 | 5.49 %                 | 5.52 %                 | 1.84                   | 8.53 %                 | 7.33 %                 | 13.40 %                | 2.27                   | 
CGRU 0.10              | 10.17 %                | 8.87 %                 | 11.92 %                | 2.08                   | 9.32 %                 | 10.37 %                | 11.83 %                | 3.65                   | 
CGRU 0.20              | 8.08 %                 | 7.16 %                 | 9.27 %                 | 2.09                   | 8.46 %                 | 8.37 %                 | 11.98 %                | 3.34                   | 
CGRU 0.23              | 7.83 %                 | 6.96 %                 | 8.94 %                 | 2.09                   | 8.27 %                 | 8.36 %                 | 11.90 %                | 3.25                   | 
CGRU 0.25              | 6.93 %                 | 6.15 %                 | 7.95 %                 | 2.12                   | 6.87 %                 | 6.92 %                 | 11.40 %                | 3.25                   | 
CGRU 0.30              | 6.60 %                 | 5.89 %                 | 7.51 %                 | 2.10                   | 6.76 %                 | 6.62 %                 | 11.17 %                | 3.09                   | 
CGRU 0.50              | 4.46 %                 | 4.04 %                 | 4.97 %                 | 2.09                   | 5.82 %                 | 4.85 %                 | 10.46 %                | 2.70                   | 
Baseline | 14.06 % | - | - | 1.54 | - | - | - | - |


# 80 ms frame length

***

## *Threshold based segmentation with "hole filling"*

 - **CGRU x.xx** : Custom Gate Recurent Unit usage with temporal weight of x.xx
 - **GRU** : The classic GRU implementation
 
model                  | Global F-measure       | Global Precision       | Global Recall          | ER                     | Class-wise F-measure   | Class-wise Precision   | Class-wise Recall      | ER                     |
---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | 
GRU                    | 4.33 %                 | 4.61 %                 | 4.08 %                 | 1.77                   | 6.53 %                 | 5.42 %                 | 9.41 %                 | 2.12                   | 
CGRU 0.10              | 11.70 %                | 13.06 %                | 10.60 %                | 1.59                   | 8.87 %                 | 10.75 %                | 6.77 %                 | 1.99                   | 
CGRU 0.20              | 9.80 %                 | 10.69 %                | 9.05 %                 | 1.65                   | 7.85 %                 | 9.15 %                 | 7.08 %                 | 2.07                   | 
CGRU 0.23              | 7.85 %                 | 8.67 %                 | 7.17 %                 | 1.67                   | 6.84 %                 | 8.49 %                 | 6.06 %                 | 2.09                   | 
CGRU 0.25              | 7.36 %                 | 8.12 %                 | 6.73 %                 | 1.68                   | 6.97 %                 | 8.34 %                 | 6.90 %                 | 2.09                   | 
CGRU 0.30              | 6.47 %                 | 6.78 %                 | 6.18 %                 | 1.76                   | 5.84 %                 | 6.97 %                 | 6.43 %                 | 2.24                   | 
CGRU 0.50              | 5.36 %                 | 5.32 %                 | 5.41 %                 | 1.87                   | 6.35 %                 | 6.11 %                 | 9.10 %                 | 2.39                   | 
Baseline | 14.06 % | - | - | 1.54 | - | - | - | - |



## *Hysteresis based segmentation*

model                  | Global F-measure       | Global Precision       | Global Recall          | ER                     | Class-wise F-measure   | Class-wise Precision   | Class-wise Recall      | ER                     |
---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | 
GRU                    | 4.86 %                 | 6.55 %                 | 3.86 %                 | 1.48                   | 8.28 %                 | 7.87 %                 | 9.51 %                 | 1.64                   | 
CGRU 0.10              | 6.41 %                 | 9.89 %                 | 4.75 %                 | 1.38                   | 4.84 %                 | 7.19 %                 | 3.29 %                 | 1.54                   | 
CGRU 0.20              | 5.82 %                 | 8.97 %                 | 4.30 %                 | 1.38                   | 4.74 %                 | 6.81 %                 | 3.86 %                 | 1.55                   | 
CGRU 0.23              | 4.96 %                 | 7.76 %                 | 3.64 %                 | 1.39                   | 4.71 %                 | 5.89 %                 | 4.38 %                 | 1.54                   | 
CGRU 0.25              | 4.48 %                 | 6.94 %                 | 3.31 %                 | 1.40                   | 4.25 %                 | 5.15 %                 | 3.89 %                 | 1.57                   | 
CGRU 0.30              | 4.73 %                 | 7.14 %                 | 3.53 %                 | 1.41                   | 5.41 %                 | 7.28 %                 | 5.09 %                 | 1.59                   | 
CGRU 0.50              | 4.55 %                 | 6.37 %                 | 3.53 %                 | 1.46                   | 6.14 %                 | 6.67 %                 | 7.23 %                 | 1.69                   | 
Baseline | 14.06 % | - | - | 1.54 | - | - | - | - |



## *Primitive based segmentation*

model                  | Global F-measure       | Global Precision       | Global Recall          | ER                     | Class-wise F-measure   | Class-wise Precision   | Class-wise Recall      | ER                     |
---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | 
GRU                    | 5.55 %                 | 6.48 %                 | 4.86 %                 | 1.60                   | 8.53 %                 | 7.37 %                 | 11.46 %                | 1.88                   | 
CGRU 0.10              | 3.81 %                 | 5.26 %                 | 2.98 %                 | 1.50                   | 3.43 %                 | 4.01 %                 | 3.97 %                 | 1.75                   | 
CGRU 0.20              | 3.86 %                 | 5.16 %                 | 3.09 %                 | 1.52                   | 4.45 %                 | 6.22 %                 | 5.04 %                 | 1.83                   | 
CGRU 0.23              | 3.77 %                 | 4.84 %                 | 3.09 %                 | 1.56                   | 4.64 %                 | 5.76 %                 | 5.55 %                 | 1.87                   | 
CGRU 0.25              | 4.48 %                 | 5.55 %                 | 3.75 %                 | 1.58                   | 5.00 %                 | 5.92 %                 | 6.96 %                 | 1.91                   | 
CGRU 0.30              | 3.33 %                 | 3.97 %                 | 2.87 %                 | 1.64                   | 3.94 %                 | 3.49 %                 | 6.56 %                 | 2.02                   | 
CGRU 0.50              | 4.24 %                 | 5.08 %                 | 3.64 %                 | 1.61                   | 6.15 %                 | 5.54 %                 | 8.60 %                 | 1.90                   |  
Baseline | 14.06 % | - | - | 1.54 | - | - | - | - |


# Different segmentation methods results for "old mel without blank" and **160** ms frame length

***

## *Threshold based segmentation with "hole filling"*

 - **CGRU x.xx** : Custom Gate Recurent Unit usage with temporal weight of x.xx
 - **GRU** : The classic GRU implementation
 
model                  | Global F-measure       | Global Precision       | Global Recall          | ER                     | Class-wise F-measure   | Class-wise Precision   | Class-wise Recall      | ER                     |
---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | 
GRU                    | 5.85 %                 | 6.70 %                 | 5.19 %                 | 1.64                   | 8.61 %                 | 7.85 %                 | 11.91 %                | 1.94                   | 
CGRU 0.10              | 7.94 %                 | 9.45 %                 | 6.84 %                 | 1.57                   | 7.93 %                 | 11.06 %                | 6.29 %                 | 2.08                   | 
CGRU 0.20              | 6.91 %                 | 8.01 %                 | 6.07 %                 | 1.61                   | 6.22 %                 | 8.59 %                 | 6.18 %                 | 2.15                   | 
CGRU 0.23              | 6.67 %                 | 7.58 %                 | 5.96 %                 | 1.64                   | 6.19 %                 | 8.22 %                 | 6.56 %                 | 2.17                   | 
CGRU 0.25              | 7.33 %                 | 8.20 %                 | 6.62 %                 | 1.65                   | 6.92 %                 | 9.20 %                 | 7.16 %                 | 2.15                   | 
CGRU 0.30              | 5.80 %                 | 6.40 %                 | 5.30 %                 | 1.69                   | 5.65 %                 | 7.10 %                 | 6.62 %                 | 2.16                   | 
CGRU 0.50              | 5.05 %                 | 5.71 %                 | 4.53 %                 | 1.67                   | 6.47 %                 | 6.82 %                 | 9.15 %                 | 2.00                   | 
Baseline | 14.06 % | - | - | 1.54 | - | - | - | - |



## *Hysteresis based segmentation*

model                  | Global F-measure       | Global Precision       | Global Recall          | ER                     | Class-wise F-measure   | Class-wise Precision   | Class-wise Recall      | ER                     |
---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | 
GRU                    | 5.97 %                 | 8.40 %                 | 4.64 %                 | 1.43                   | 9.51 %                 | 9.95 %                 | 11.05 %                | 1.61                   | 
CGRU 0.10              | 4.36 %                 | 7.39 %                 | 3.09 %                 | 1.35                   | 4.07 %                 | 6.28 %                 | 3.45 %                 | 1.53                   | 
CGRU 0.20              | 3.91 %                 | 6.70 %                 | 2.76 %                 | 1.35                   | 3.68 %                 | 5.44 %                 | 3.63 %                 | 1.51                   | 
CGRU 0.23              | 3.56 %                 | 5.96 %                 | 2.54 %                 | 1.37                   | 3.56 %                 | 5.53 %                 | 3.79 %                 | 1.55                   | 
CGRU 0.25              | 3.69 %                 | 6.09 %                 | 2.65 %                 | 1.37                   | 3.69 %                 | 5.42 %                 | 4.18 %                 | 1.57                   | 
CGRU 0.30              | 4.27 %                 | 6.93 %                 | 3.09 %                 | 1.37                   | 4.90 %                 | 6.96 %                 | 5.27 %                 | 1.54                   | 
CGRU 0.50              | 5.11 %                 | 7.98 %                 | 3.75 %                 | 1.38                   | 7.72 %                 | 9.36 %                 | 8.53 %                 | 1.50                   | 
Baseline | 14.06 % | - | - | 1.54 | - | - | - | - |



## *Primitive based segmentation*

model                  | Global F-measure       | Global Precision       | Global Recall          | ER                     | Class-wise F-measure   | Class-wise Precision   | Class-wise Recall      | ER                     |
---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | ---------------------- | 
GRU                    | 7.05 %                 | 9.12 %                 | 5.74 %                 | 1.48                   | 11.40 %                | 11.42 %                | 13.84 %                | 1.65                   | 
CGRU 0.10              | 2.69 %                 | 4.76 %                 | 1.88 %                 | 1.35                   | 3.05 %                 | 5.48 %                 | 4.27 %                 | 1.59                   | 
CGRU 0.20              | 3.30 %                 | 5.16 %                 | 2.43 %                 | 1.41                   | 4.18 %                 | 5.28 %                 | 6.12 %                 | 1.65                   | 
CGRU 0.23              | 3.88 %                 | 5.98 %                 | 2.87 %                 | 1.41                   | 5.33 %                 | 8.05 %                 | 6.92 %                 | 1.65                   | 
CGRU 0.25              | 4.14 %                 | 6.26 %                 | 3.09 %                 | 1.42                   | 5.54 %                 | 7.61 %                 | 7.23 %                 | 1.67                   | 
CGRU 0.30              | 4.82 %                 | 7.13 %                 | 3.64 %                 | 1.43                   | 6.35 %                 | 8.15 %                 | 8.51 %                 | 1.68                   | 
CGRU 0.50              | 6.29 %                 | 8.59 %                 | 4.97 %                 | 1.46                   | 8.64 %                 | 9.08 %                 | 12.33 %                | 1.69                   | 
Baseline | 14.06 % | - | - | 1.54 | - | - | - | - |


model                  | Global F-measure       | ER   |
---------------------- | ---------------------- | ---- |
GRU                    | 3.79 %                 | 2.38 |
CGRU 0.10              | 6.61 %                 | 2.88 |
CGRU 0.20              | 9.60 %                 | 2.70 |
CGRU 0.23              | 10.41 %                | 2.62 |
CGRU 0.25              | 10.59 %                | 2.59 |
CGRU 0.30              | 9.45 %                 | 2.54 |
CGRU 0.50              | 4.23 %                 | 2.52 |
Baseline               | 14.06 %                | 1.54 |